# Heading


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path ='/home/nrw/Documents/projects_Spring2018/howe299r/Experiments/26Mar2018/data/deltas/'
viz = '03April_ID1.csv'
imu = '03IMU.csv'
vizf = pd.read_csv(path+viz,sep='\t')
vizf.dropna(how="all", inplace=True) 
imuf = pd.read_csv(path+imu,sep='\t')
imuf.dropna(how="all", inplace=True) 

yaw, pitch, roll = vizf["yaw"], vizf["pitch"], vizf["roll"]
yaw, pitch, roll = vizf["yaw"]*57.29578, vizf["pitch"]*57.29578, vizf["roll"]*57.29578
yaw, pitch, roll = yaw, pitch, roll 
xIMU,yIMU,zIMU = imuf["x"], imuf["y"], imuf["z"]

forces = np.arange(0,yaw.shape[0]*10,10)

#print(len(yaw))
#print(len(yIMU))
#print(yIMU)
#print(roll)
#print(pitch)
#print(len(forces))

KeyError: 'yaw'

In [2]:
yIMU =  [-0.3125, -1.4375, -2.4375, -3.25,   -3.75,   -4.     ,-5.625  ,-6.    ]
yIMU += [-0.4375 ,-1.0625 ,-1.875  ,-2.6875 ,-3.75   ,-4.5625 ,-5.8125 ,-6.125 ]
yIMU += [-0.625  ,-1.5    ,-2.125  ,-2.875  ,-4.125  ,-4.4375 ,-5.75   ,-6. ]
yaw =  [ 0.29166667 ,-0.0483631  ,-0.26339286 ,-0.29092262 ,-0.00595238  ,0.52901786
 ,-0.3110119   ,0.09895833]
yaw += [-1.17708333  ,0.06324405  ,0.11607143  ,0.16889881 ,-0.02827381  ,0.02455357
 ,-0.36011905  ,0.19270833]  
yaw += [ 0.01041667 ,-0.06622024  ,0.10714286  ,0.15550595 ,-0.29613095  ,0.18973214
 ,-0.32440476  ,0.22395833]

print(len(yIMU))
print(len(yaw))
plt.scatter(forces, yIMU, label='imu')
plt.scatter(forces, yaw, label='aprilTags') #????
plt.ylabel('degrees')
plt.xlabel('force (g)')
plt.legend()
plt.show()

24
24


NameError: name 'plt' is not defined

In [4]:
import numpy as np
import plotly.plotly as py
import plotly.offline as po
import plotly.graph_objs as go
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

po.init_notebook_mode(connected=True)

trace0 = go.Scatter( x = forces, y = yIMU, mode = 'markers',
    name = 'degrees (by IMU)' )

trace1 = go.Scatter( x = forces, y = yaw, mode = 'markers',
    name = 'true degrees (by apriltag)' )


forces = forces.reshape(-1, 1)
myX = forces
myy = yIMU 
#regr= Ridge(fit_intercept=True, alpha=1.0, random_state=0, normalize=True)
regr = linear_model.LinearRegression()
regr.fit(myX, myy)
coef_ridge = regr.coef_
gridx = np.linspace(myX.min(), myX.max(), 20)
coef_ = regr.coef_ * gridx + regr.intercept_
yPred= regr.predict(myX) 
#plt.plot(gridx, coef_, 'g-', label="ridge regression")

trace2 = go.Scatter( x= gridx, y = coef_,
    name = 'linear fit (w/ridge penalty)' )
           

In [5]:
data = [trace1, trace0, trace2]

layout = go.Layout(
    title='Force vs Degrees of Deflection',
    yaxis=dict(title='degrees'),
    xaxis=dict(title='Force (in grams)'),
    legend=dict(x=.1, y=-.5) )

fig = go.Figure(data=data, layout=layout)
# Plot and embed in ipython notebook!

print('Coefficients: \n', regr.coef_)
print("Mean squared error: %.2f" % mean_squared_error(yIMU, yPred))
print('Variance score (ideal 1): %.2f' % r2_score(yIMU, yPred)) 
po.iplot(fig)
#po.plot(fig, filename='temp_plot.html')

Coefficients: 
 [-0.07983631]
Mean squared error: 0.04
Variance score (ideal 1): 0.99


In [6]:
resid = yIMU - yPred
layout = go.Layout(
    title='Force vs Degrees of Deflection',
    yaxis=dict(title='degrees'),
    xaxis=dict(title='Force (in grams)'),
    legend=dict(x=.1, y=-.5) )
traceResid = go.Scatter( x = yIMU, y = resid, mode = 'markers',
    name = 'resid' )
data = [traceResid]
layout = go.Layout(
    title='Residuals',
    yaxis=dict(title='residuals'),
    xaxis=dict(title='yIMU (degrees)'),
    legend=dict(x=.1, y=-.5) )

fig = go.Figure(data=data, layout=layout)
po.iplot(fig)

print(yIMU)
print(resid)

0   -0.6250
1   -1.5000
2   -2.1250
3   -2.8750
4   -4.1250
5   -4.4375
6   -5.7500
7   -6.0000
Name: y, dtype: float64
0    0.010417
1   -0.066220
2    0.107143
3    0.155506
4   -0.296131
5    0.189732
6   -0.324405
7    0.223958
Name: y, dtype: float64
